# Making allele frequency plots

This notebook contains code to process the CRISPResso "allele frequency table" files from base editor validation experiments. The input is are 2 input files: the first contains metainformation about each sample to make the "allele frequency" file, and the second contains metainformation to compute correlations between the log-normalized read counts. The output of this file are 3 files for each sgRNA / primer pair: 
1. a file containing all alleles and their read counts for each sample
2. a filtered version of (1) that only contains alleles with at least 1% abundance in any sample
3. a file containing the Pearson correlations between log-normalized read counts of each allele with > 100 reads in at least one sample

(1) is the starting file used to show the abundance of specific edits over time (code in BEV_aa_over_time.ipynb). (2) is the starting file used to create allele-level heatmaps (this was done using GraphPad Prism). (3) is the starting file for the plots showing the replicate correlation for validation experiments (actual plots were made using GraphPad Prism).

In [25]:
import pandas as pd 
import numpy as np 
import base_edit_functions_v3 as be
from math import log
from os import path
import sys, itertools

In [26]:
print('Python version: ' + sys.version)

Python version: 2.7.9 |Continuum Analytics, Inc.| (default, Dec 15 2014, 10:37:34) 
[GCC 4.2.1 (Apple Inc. build 5577)]


In [27]:
modules = ['pandas', 'numpy']
for module in modules:
    try:
        print(module + ' ' + sys.modules[module].__version__)
    except:
        print(module + ' has no __version__ attribute')

pandas 0.23.4
numpy 1.16.2


In [28]:
'''
Read in first metainformation file

COLUMNS
-------

sg : sg number (as referenced in paper / supplementary tables)
sgRNA_sequence : sequence of sg
BEV_start : BEV number for first sample in sg
BEV_end : BEV number for last sample in sg
primer : name of primer pair used to amplify genomic locus
offset : frame for translation (manually determined for each sg / primer pair)
rev_com : whether or not to reverse complement the Aligned_Sequence prior to translation (manually determined for each sg / primer pair)
BEV_ref : reference sample for log-fold change (LFC) calculation; if multiple BEV numbers are given, these are replicates that will be averaged
BEV_test : test sample for LFC calculation; if multiple BEV numbers are given, these are replicates that will be averaged

'''

input_file = pd.read_csv('../Data/Validation_CRISPResso_results/allele_freq/BEV_allele_freq_input_v3.csv')
input_file.head()

,sg,sgRNA_sequence,BEV_start,BEV_end,primer,offset,rev_com,BEV_ref,BEV_test
0,sg31,GGTGGCCTACAGTCTGCTCA,114,123,PARP1_F3_PARP1_R3,1,True,116;117,118;119
1,sg31,GGTGGCCTACAGTCTGCTCA,114,123,PARP1_F3_PARP1_R3,1,True,120;121,122;123
2,sg31,GGTGGCCTACAGTCTGCTCA,7,8,PARP1_F3_PARP1_R3,1,True,7;8,7;8
3,sg32,CAGGATCCCAGCAAAGTTGG,124,149,PARP1_F2_PARP1_R2,1,True,126;127,128;129
4,sg32,CAGGATCCCAGCAAAGTTGG,124,149,PARP1_F2_PARP1_R2,1,True,126;127,130;131


In [29]:
'''
Read in correlation metainformation file

COLUMNS
-------
sg : sg number (as referenced in paper / supplementary tables)
reps_for_correlation : semicolon-separated BEV numbers of which to calculate the pairwise Pearson correlation of the log-normalized read counts

'''

corr_input = pd.read_csv('../Data/Validation_CRISPResso_results/allele_freq/BEV_allele_corr_input_v2.csv')
corr_input.head()

,sg,reps_for_correlation
0,sg24,348;349
1,sg24,350;351
2,sg25,352;353
3,sg25,354;355
4,sg26,356;357


In [30]:
# This function converts the BEV number from an int to a 3-digit string
def get_bev_str(bev):
    bev = int(bev)
    if bev < 10:
        return '00'+str(bev)
    if bev < 100:
        return '0'+str(bev)
    return str(bev)

# This function returns False for any rows (alleles) that have a value < the given value in ALL of the given cols
def read_count_filter(row,cols,val):
    for col in cols:
        if row[col] > val:
            return True
    return False

# This function returns the tranlation of a given sequence and frame
def translate(seq, frame, codon_map):
    aa = ''
    i = frame - 1
    while i < len(seq):
        substring = ''
        while len(substring) < 3:
            if i < len(seq):                
                if seq[i] != '-':
                    substring += seq[i]
                i += 1
            else:
                break
        if len(substring) == 3:
            if 'N' in substring:
                aa = aa + '-'
            else:
                aa = aa + codon_map[substring]
    return aa

codon_map = {'TTT':'F', 'TTC':'F', 'TTA':'L', 'TTG':'L', 'CTT':'L', 'CTC':'L', 'CTA':'L', 'CTG':'L', 'ATT':'I', 'ATC':'I',
             'ATA':'I', 'ATG':'M', 'GTT':'V', 'GTC':'V', 'GTA':'V', 'GTG':'V', 'TCT':'S', 'TCC':'S', 'TCA':'S', 'TCG':'S',
             'CCT':'P', 'CCC':'P', 'CCA':'P', 'CCG':'P', 'ACT':'T', 'ACC':'T', 'ACA':'T', 'ACG':'T', 'GCT':'A', 'GCC':'A',
             'GCA':'A', 'GCG':'A', 'TAT':'Y', 'TAC':'Y', 'TAA':'*', 'TAG':'*', 'CAT':'H', 'CAC':'H', 'CAA':'Q', 'CAG':'Q',
             'AAT':'N', 'AAC':'N', 'AAA':'K', 'AAG':'K', 'GAT':'D', 'GAC':'D', 'GAA':'E', 'GAG':'E', 'TGT':'C', 'TGC':'C',
             'TGA':'*', 'TGG':'W', 'CGT':'R', 'CGC':'R', 'CGA':'R', 'CGG':'R', 'AGT':'S', 'AGC':'S', 'AGA':'R', 'AGG':'R',
             'GGT':'G', 'GGC':'G', 'GGA':'G', 'GGG':'G'}

# This function merges together the "Allele_frequency_table_around_sgRNA" files
def merge_bev_file(filepath,bev,sg_seq,existing_df,cols):
    if not path.exists(filepath):
        print 'no file'
        return existing_df,cols
    df = pd.read_table(filepath,index_col=False)
    # Sum together any rows that share both 'Aligned Sequence' and 'Reference Sequence' with each other (this is rare)
    df_summed = df[['Aligned_Sequence','Reference_Sequence','#Reads','%Reads']].groupby(['Aligned_Sequence','Reference_Sequence'],as_index=False).agg('sum')
    cols.append(str('_BEV_'+str(bev)))
    df_summed = df_summed.rename(columns={'#Reads':str('#Reads_BEV_'+str(bev)),'%Reads':str('%Reads_BEV_'+str(bev))})
    # Outer merge onto existing dataframe
    merge = pd.merge(existing_df,df_summed,how='outer',on=['Aligned_Sequence','Reference_Sequence'])
    # Fill in nans with 0
    merge = merge.fillna(0)
    return merge,cols

def check_filepath(filepath,bev,primer,sg_seq):
    file_loc = filepath+'/CRISPResso_on_'+bev+'_'+primer+'/'+'Alleles_frequency_table_around_sgRNA_'+sg_seq+'.txt'
    if path.exists(file_loc):
        return file_loc
    return ''
        
# This function gets the filepath from Base Editing shared drive
def get_path(bev_num,primer,sg_seq):
    bev = 'BEV_' + get_bev_str(bev_num)
    filepath = '../Data/Validation_CRISPResso_results/Completed_CRISPResso_files_v2'
    return check_filepath(filepath,bev,primer,sg_seq)

# This function gets and merges all "Allele_frequency_table_around_sgRNA" files for a given sgRNA sequence
# (i.e. different replicates, drug conditions, etc.)
# This is customized to work with files with the "BEV" notation, stored on the Base Editing shared drive
def get_bev_files(bev_list):
    merge = pd.DataFrame(columns=['Aligned_Sequence','Reference_Sequence'])
    cols = []
    for bev,sg_seq,primer_name in bev_list:
        filepath = get_path(bev,primer_name,sg_seq)
        if filepath != '':
            merge,cols = merge_bev_file(filepath=filepath,bev=bev,sg_seq=sg_seq,existing_df=merge,cols=cols)
    return merge,cols

# This function returns True if the allele is unedited (i.e. WT) and False otherwise
def get_wt_col(row):
    if row['Aligned_Sequence'] == row['Reference_Sequence']:
        return True
    else:
        return False

'''
This function calculates the LFC

INPUTS
------
row : row of metainformation input file containing BEV_test and BEV_ref columns
data_file : merged dataframe containing log-normalized rpm for each allele

OUTPUTS
-------
data_file : merged dataframe, now with LFC columns for each BEV test / ref pair

'''
def get_lfc_v2(row,data_file):
    cols = []
    bev_list = row['BEV_test'].split(';')
    
    # Go through each test sample in BEV_test column
    for i,bev in enumerate(bev_list):
        test = get_bev_str(bev)
        
        # Get reference sample for LFC from BEV_ref column
        ref = get_bev_str(row['BEV_ref'].split(';')[i])
        
        # Calculate LFC
        data_file['LFC_'+test+'-'+ref] = data_file['#Reads_BEV_'+test+';lognorm'] - data_file['#Reads_BEV_'+ref+';lognorm']  
        cols.append('LFC_'+test+'-'+ref)
        
    # Average together LFC columns
    data_file['AvgLFC_'+'_'.join(bev_list)] = data_file.loc[:,cols].mean(axis=1)
    return data_file

def process_data_v2(data):
    cols_to_use = []
    bev_list = []
    for i,row in data.iterrows():
        for bev in range(row['BEV_start'],row['BEV_end']+1):
            bev_list.append((get_bev_str(bev),row['sgRNA_sequence'],row['primer']))

    merge,cols = get_bev_files(bev_list)
    
    # Calculate log-normalized reads per million for each col
    for col in ['#Reads'+col for col in cols]:
        colsum = merge[col].sum()
        merge.loc[:,str(col+';lognorm')] = merge[col].apply(lambda x: log((float(x)/float(colsum))*1000000 + 1,2))
        
    # Apply read count filter
    merge.loc[:,'%read_count_filter'] = merge.apply(read_count_filter,args=(['%Reads'+col for col in cols],1),axis=1) # less than 1% of all reads
    merge.loc[:,'#read_count_filter'] = merge.apply(read_count_filter,args=(['#Reads'+col for col in cols],100),axis=1) # less than 100 reads
    
    # Reverse complement if necessary
    if row['rev_com']:
        merge.loc[:,'Aligned_Sequence'] = merge.loc[:,'Aligned_Sequence'].apply(be.revcom)
        merge.loc[:,'Reference_Sequence'] = merge.loc[:,'Reference_Sequence'].apply(be.revcom)
        
    # Translate Aligned_Sequence to amino acids
    merge.loc[:,'Translated'] = merge.loc[:,'Aligned_Sequence'].apply(translate,args=(row['offset'],codon_map,))
    return merge

def get_corr_name(row):
    cols = [row['R1'],row['R2']]
    cols.sort()
    return '_'.join(cols)

def get_correlation_cols(corr_input,sg):
    corr_input = corr_input.loc[corr_input['sg'] == sg,:]
    combos = []
    for i,r in corr_input.iterrows():
        bevs = r['reps_for_correlation'].split(';')
        bevs = ['#Reads_BEV_'+get_bev_str(bev)+';lognorm' for bev in bevs]
        if len(bevs) > 1:
            combos.extend(itertools.combinations(bevs,2))
    combos = ['_'.join(combo) for combo in combos]
    return combos

def run(input_file,corr_input):

    sg_list = list(set(input_file['sg'].tolist())) # drop duplicates from list

    # Go through each sgRNA separately    
    for sg in sg_list:
        print sg
        
        # Filter input file to contain only rows for given sgRNA
        data = input_file.loc[input_file['sg'] == sg,:]
        
        # Merge all the allele read counts
        # To do this, drop the BEV_test and BEV_ref columns (which just have information needed for the LFC calculation)
        # Then drop duplicate rows
        data_dedup = data.drop_duplicates(subset=['sg','BEV_start','BEV_end','sgRNA_sequence','primer','offset','rev_com'])
        merge = process_data_v2(data_dedup)
        
        # Get the WT column
        merge['WT'] = merge.apply(get_wt_col,axis=1)
        
        # Now, go through each row and calculate the LFC for each of the pairs specified in BEV_test and BEV_ref
        for i,r in data.iterrows():               
            merge = get_lfc_v2(r,merge)
            
        # Write out 2 files: full file (merge) and filtered file (only including alleles with > 1% reads in at least one condition)
        merge.to_csv('../Data/Validation_CRISPResso_results/allele_freq/'+str(r['sg'])+'_'+r['primer']+'_allele_frequency_table_around_sgRNA.csv',index=False)
        filtered = merge[merge['%read_count_filter'] == True]
        filtered.to_csv('../Data/Validation_CRISPResso_results/allele_freq/'+str(r['sg'])+'_'+r['primer']+'_filtered_allele_frequency_table_around_sgRNA.csv',
                        index=False)            

        # Get correlations matrix of log-normalized rpm, using only alleles with > 100 reads in at least one sample
        merge = merge[merge['#read_count_filter'] == True]
        cols = [x for x in list(merge) if 'lognorm' in x]
        correlations = merge[cols].corr(method='pearson')
        correlations['R1'] = correlations.index
        correlations = correlations.melt(id_vars = 'R1', value_vars=list(correlations).remove('R1'),var_name='R2',value_name='Pearson')
        
        # Drop correlations that are not specified in corr_input
        correlations['Reps'] = correlations.apply(get_corr_name,axis=1)
        combos = get_correlation_cols(corr_input,sg)
        correlations = correlations[correlations['Reps'].isin(combos)]
        
        # Drop duplicate rows (i.e. A vs B and B vs A)
        correlations = correlations.drop_duplicates(subset=['Reps'])
        
        # Write to file
        correlations.to_csv('../Data/Validation_CRISPResso_results/allele_freq/corr_plots/#read_count_filter_pearson/sg'+str(r['sg'])+'_'+r['primer']+'_correlations.csv')
        
    return 
    

Now, we run the two input files and produce allele tables for all sgRNAs.

In [31]:
run(input_file,corr_input)

sg15
sg14
sg17
sg16
sg11
sg10
sg13
sg12
sg19
sg18
sg36
sg35
sg34
sg33
sg32
sg31
sg30
sg9
sg8
sg1
sg3
sg2
sg5
sg4
sg7
sg6
sg28
sg29
sg24
sg25
sg26
sg27
sg20
sg21
sg22
sg23
